## Problem instance

In [1]:
from __future__ import annotations

import itertools
import random

# General imports
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

from more_itertools import distinct_permutations

# rustworkx graph library
import rustworkx as rx
from rustworkx.visualization import mpl_draw

# SciPy minimizer routine
from scipy.optimize import minimize

In [1]:
def define_instance(n, instance, verbose):
    """
    Parameters
    ----------
        n (int): instance dimension.
        instance(int): number of the instance.
        verbose (bool): if True, print is activated.

    Return
    ------
        U (set): set Universe of the instance.
        subsets_dict (dict): dictionary that enumerates subsets of the instance.
    """
    if n==6:
        # ############# DIM 6 #############
        all_instances = {
        1: [{4, 6, 7, 9, 10, 11}, {1, 2, 5, 6, 11, 12}, {8, 1, 12}, {2, 3, 5}, {1, 3, 4, 5, 9, 12}, {2, 6, 7, 9, 12}] ,
        2: [{2, 11, 12, 6}, {2, 4, 6, 8, 9, 11}, {1, 3, 5, 7, 10, 12}, {2, 7}, {2, 3, 4, 5, 8, 12}, {1, 2, 8, 9, 12}] ,
        3: [{8, 10, 3}, {2, 4, 5, 6, 9, 11, 12}, {1, 3, 7, 8, 10}, {3, 4, 6, 8, 11}, {2, 3, 4, 6, 7, 9, 12}, {1, 7}] ,
        4: [{1, 4, 5, 6, 8, 11, 12}, {3, 6, 8, 9, 10}, {1, 2, 11, 5}, {2,3, 7, 9,10}, {8, 3, 12, 6}, {4, 7, 12}] ,
        5: [{11, 7}, {1, 2, 4, 5, 9, 11}, {1, 3, 4, 6, 8}, {2, 5}, {3, 6, 7, 8,10, 12}, {9, 10, 12}] ,
        6: [{2, 3, 4, 5, 7, 8,9, 10, 11, 12}, {5, 9, 10, 11}, {3, 6, 7, 8}, {1, 2, 4, 12}, {1, 6}, {2, 3, 4, 7, 8, 12}] ,
        7: [{1, 2, 5, 6, 7, 9, 12}, {10, 3, 4}, {2, 5, 7, 8, 9, 10, 11}, {8, 11}, {1, 7, 8, 11, 12}, {11, 9, 3, 7}] ,
        8: [{1, 2, 4, 5, 6, 8, 9, 10}, {1, 4, 6, 7}, { 5, 8, 9, 11, 12}, {4, 7, 8, 9, 10, 11, 12}, {2, 3, 4, 5, 6, 7, 11, 12}, {10, 2, 3}] ,
        9: [{1, 4, 7, 9, 10, 11, 12}, {2, 3, 5, 6, 8}, {1, 2, 4, 9, 12}, {2,3, 6, 7, 8, 9}, {1, 4, 6, 7, 8, 10, 11}, {8, 11, 3, 6}] ,
        10: [{1, 11, 12, 6}, {2, 5, 8, 10, 11, 12}, {2, 6, 7, 8, 9, 10, 11}, {2, 3,4, 5, 7, 8, 9, 10}, {1, 2, 4, 5, 6, 7, 8, 11}, {1, 5, 6, 9, 10, 11, 12}]} 

    elif n==8:
        ########### DIM 8 ###########
        all_instances = { 
        1: [{1, 2, 3, 4, 7, 8, 9, 12, 13, 14}, {1, 4, 5, 8, 9, 12, 13, 14, 16}, {1, 6, 9, 10, 11, 13, 15, 16}, {5, 6, 10, 11, 15, 16}, {1, 6, 7, 8, 9, 10, 14, 15}, {8, 13, 4, 12}, {2, 3, 4, 5, 7, 8, 11, 12, 13}, {4, 5, 6, 7, 10, 12, 14, 16}],
        2: [{1, 2, 3, 5,7,12, 15}, {6, 7, 8, 9}, {4, 6, 8, 9, 10, 11, 13, 14, 16}, {3, 4, 5, 6, 7, 9, 10, 11, 12, 15}, {4, 12, 14}, {2, 3, 7, 8, 10, 15}, {1, 4, 6, 8, 9, 10, 12, 14, 15}, {1, 2, 3, 5, 10, 11,13, 15, 16}],
        3: [{4, 7, 14, 15,16}, { 4, 5, 8, 9, 10, 13, 14, 16}, {1, 3, 4, 6, 8, 9, 10, 12, 13, 14}, {1, 3, 5, 6, 10}, {2, 4, 6, 9, 10, 11, 15}, {1, 2, 4, 5, 9, 13, 14, 15}, {1, 2, 3, 6, 7, 11, 12, 15}, {2, 8, 9, 11, 12, 13}], 
        4: [{2, 4, 5, 6, 9}, {13, 14, 15, 16}, {1, 4, 5, 6}, { 7, 8, 10, 12}, {1, 3, 7, 8, 10, 11, 12, 13, 14, 15, 16}, {2, 3, 9, 11}, {3, 4, 5, 7, 8, 10, 12}, {3, 7, 8, 9, 10, 11, 14}],
        5: [{5, 8, 10, 11, 12, 13, 16}, {1, 2, 3, 4, 6, 7, 9, 14, 15}, {1, 2, 5, 7, 8, 9, 10, 11, 12, 13, 15, 16}, {3, 4, 9, 11, 13, 14, 16}, {2, 3, 6, 10, 13, 16}, {1, 3, 8, 10, 11, 13,14, 15}, {1, 6, 8, 10, 11, 13, 14, 16}, {3, 4, 7, 10, 11, 13, 14, 15, 16}],
        6: [{1, 2, 5, 6, 7, 10, 12, 15}, {5, 6, 9, 10}, {2, 7, 11, 12}, {3, 6, 7, 8, 9, 12, 15, 16}, {1, 3, 4, 8, 13, 14, 15, 16}, {2, 3, 6, 7, 8, 10, 15, 16}, {1, 2, 3, 4, 7, 8, 12, 13, 14}, {2, 3, 8, 11, 12, 14}],
        7: [{2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 15}, {2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16}, {10, 11, 15, 7}, {2, 3, 9, 10, 11, 14, 16}, {1, 2, 7, 10, 11, 12, 13, 14, 15}, {1, 4, 9, 14, 16}, {3, 4, 5, 9, 11, 12, 14, 16}, {2, 7, 9, 10, 11, 12, 13, 14, 15, 16}],
        8: [{1, 4, 8, 9, 10, 13, 14, 15, 16}, {1, 2, 4, 5, 6, 7, 9, 12, 13}, {2, 3, 5, 6, 7, 11, 12}, { 5, 11, 12}, {1, 2, 3, 7, 8, 10, 11, 13}, {1, 2, 4, 7, 8, 13, 14, 15, 16}, {2, 3, 6, 7}, {1, 3, 4, 6, 9, 11, 12, 14, 16}],
        9: [{2, 4, 14,15}, {1, 3, 6, 8}, {5, 7, 10, 13, 16}, {1, 2, 4, 6, 7, 10, 11, 13, 15, 16}, {4, 6, 7, 11, 12, 13, 15}, {3, 5, 8, 9, 12, 14}, {9, 11, 12}, {2, 3, 4, 5, 7, 8, 9, 10, 12, 14, 15, 16}],
        10: [{2, 3, 5, 6, 7, 8, 9, 11, 13, 16}, {1, 4, 5, 7, 9, 12, 14}, {6, 7, 11, 13, 14, 15, 16}, {6, 7, 8, 9, 11, 15, 16}, {2, 5, 10, 12, 14}, {1, 5, 6, 7, 11, 12, 14}, {2, 3, 4, 5, 7, 11, 12, 13, 14, 15, 16}, {1, 3, 4, 13}]
        }
    elif n==10:
            ########## DIM 10 ###########
        all_instances = {
         1:[{1, 4, 7, 8, 9, 12, 13, 14, 15, 17}, {1, 3, 6, 8, 9, 13, 14, 16, 17}, 
                         {2, 4, 5, 7, 10,11}, {1, 4, 5, 8, 9, 11, 13, 14, 17, 19}, {1, 5, 6, 10, 11, 13, 14, 16, 18, 19, 20}, 
                         {1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 16}, {2, 3, 5, 6, 10, 11, 16, 18,19,20}, {12, 15, 18, 19,20}, 
                         {2, 4, 5, 6, 8, 9, 10, 11, 13, 15, 17, 19, 20}, {5, 7, 8, 10, 12, 13, 16, 17, 18, 19, 20}],                 
        2:[{1, 2, 5, 7, 8, 9, 10, 11, 12, 13, 15, 17, 20}, {2, 3, 4, 5, 9,10,20}, {1, 6, 7, 8, 11, 12, 13, 19}, { 14, 15, 16, 17, 18}, 
                         {3, 5, 6, 10, 16, 18, 20}, {4, 7, 8, 9, 12, 14, 17, 19}, {3, 4, 6, 14, 16, 18, 19}, {1, 2, 11, 13, 15}, 
                         {2, 6, 7, 10, 13, 14, 16, 17, 20}, {2, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 20}],                
        3:[{2, 3, 6, 7, 9}, {4, 5, 8, 12, 15}, {10, 11, 13, 14, 17}, {1, 16, 18, 19, 20}, {2, 10, 11, 13, 15, 16, 17, 18}, {2, 5, 18,19,20}, 
                         {3, 4, 5, 6, 7, 8, 10, 11, 16, 17, 20}, {7, 8, 10, 11, 13, 14, 16, 17}, {1, 3, 4, 6, 9, 12, 15}, 
                         {1, 2, 4, 5, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20}],                 
        4:[{1, 2, 5, 8, 10, 11, 13, 15,18, 20}, {3, 4, 6, 7, 9, 12, 14, 16, 17, 19}, {2, 5, 6, 11, 15, 17}, {1, 3, 4, 7, 8, 9, 10, 12}, 
                         {13, 14, 16, 18, 19, 20}, {2, 5, 6, 8, 9, 10, 11, 12, 14, 16, 18, 19, 20}, {1, 2, 6, 8, 10, 11, 13, 14, 15, 16, 18, 19, 20},
                         {1, 2, 3, 4, 7, 10, 16, 17}, {1, 2, 4, 5, 7, 8, 9, 11, 13, 14, 15, 17, 19}, {1, 2, 3, 4, 8, 9, 13, 15, 16, 17, 19}],                 
        5:[{2, 4, 5, 6, 9, 15, 16, 19}, {3, 7, 8, 12, 13, 14, 17}, {1, 10, 11, 18, 20}, {2, 8, 9, 10, 12, 13, 14, 15, 16, 17}, 
                         {1, 3, 4, 5, 6, 7, 11, 18, 19, 20}, {1, 3, 6, 7, 8, 9, 12, 14, 16, 19, 20}, {1, 4, 6, 10, 11, 12, 18, 19}, 
                         {1, 3, 4, 5, 7, 8, 9, 11, 12, 13, 15, 16, 20}, {1, 2, 9, 11, 15}, {2, 3, 4, 5, 6, 7, 9, 10, 11, 14, 19}],                 
        6:[{1, 2, 3, 4, 9, 10, 12, 13, 15, 19, 20}, {2, 3, 5, 6, 8, 10, 12, 17}, {1, 4, 7, 9, 14, 15, 19}, {11, 13, 16,18, 20}, 
                         {2, 3, 4, 5, 6, 8, 9, 13, 14, 15,16, 17}, {1, 7, 10, 11, 12, 18, 19, 20}, {1, 2, 5, 11, 13, 14, 19, 20}, 
                         {1, 5, 7, 9, 10, 11, 13}, {12, 14, 18, 19, 20}, {2, 3, 4, 6, 8, 15, 16, 17}],
        7:[{2, 3, 6, 9, 10, 12, 14, 17, 20}, {1, 4, 10, 11, 12, 14, 15, 17, 18, 20}, {2, 4, 5, 7, 8, 11, 12, 15, 17}, 
                         {2, 3, 4, 5, 6, 8, 9, 11, 14, 17, 18, 20}, {1, 2, 4, 16, 17, 18, 20}, {1, 4, 5, 7, 8, 11, 13, 15, 16, 18, 19}, 
                         {5, 6, 8, 10, 11, 13, 15}, {3, 7, 9, 12, 14, 19}, {1, 2, 4, 6, 7, 8, 9, 12, 13, 15, 17, 18, 20}, 
                         {1, 5, 10, 12, 15, 17, 18, 19}],                 
        8:[{ 4, 8, 12, 15, 16, 17, 18}, {1, 2, 4, 7, 8, 10, 11, 12, 13}, {3, 5, 6, 12, 14, 18, 19,20}, {6, 7, 9, 10, 13, 14, 19}, 
                         {1, 2, 3, 5, 11, 20}, {1, 2, 4, 7, 8, 9, 10, 11, 13, 15, 16, 17}, {1, 3, 6, 8, 10, 15, 17, 19}, 
                         {2, 3, 4, 10, 12, 13, 14, 16, 17, 20}, {2, 3, 4, 8, 11, 12, 13, 18, 20}, {1, 6, 7, 9, 10, 11, 13, 14, 16}],
        9:[{2, 4, 5, 6, 7, 11, 15, 17}, {5, 9, 10, 14, 17, 18, 20}, {4, 5, 6, 8, 9, 11, 13, 14, 16, 17, 20}, {4, 7, 8, 11,13, 15, 19}, 
                         {2, 5, 6, 7, 12, 14, 15, 16, 17}, {1, 2, 3, 6, 12, 16}, {1, 2, 5, 6, 8, 10, 14, 16, 18, 19, 20}, 
                         {1, 3, 4, 8, 9, 10, 11, 13, 18, 19,20}, {1, 4, 5, 8, 9, 10, 11, 13, 14, 16, 18}, {1, 3, 6, 7, 8, 12, 13, 15, 16, 17, 19}],
        10:[{1, 2, 6, 7, 9, 11, 18}, {3, 4, 8, 10, 13, 19, 20}, { 5, 12, 14, 15, 16, 17}, {1, 2, 5, 6, 7, 9, 11, 12, 14, 15, 16, 17, 18}, 
                         {2, 4, 7, 8, 9, 10, 12, 14, 16, 18}, {1, 4, 7, 8, 10, 13, 18}, {2, 3, 5, 8, 9, 11, 12, 16, 20}, 
                         {1, 4, 7, 8, 11, 12, 13, 14, 18, 19, 20}, {3, 4, 5, 9, 11, 12, 13, 14, 15, 16, 17}, {2, 5, 7, 8, 13, 18, 19, 20}]
        }

    # Create a dictionary containing the subsets of the instance.
    subsets_dict = dict([(k,v) for k,v in enumerate(all_instances[instance])])
    
    # Create a list of the subsets.
    subsets = all_instances[instance]
    
    # Find U set by union of the subsets.
    U = find_U_from_subsets(subsets_dict)
    

    if verbose:
        print(f"## n chosen: {n} ##")
        print(f"## instance chosen: {instance} ##")
        print("subsets:\n", subsets)
        print("U:\n", U)
    
    return U, subsets_dict

In [2]:
def build_instance_graph(subsets, verbose=False, draw_graph=False):
    """
    Parameters
    ----------
        subsets (list of sets): list of sets defining the instance.
        verbose (bool): if True, print is activated.

    Return
    ------
        list_of_intersections (list of lists): list containing lists of intersections 
                                               for each subset of the instance, i.e. lists containing 
                                               all the connections each node of the instance has.
    
    """
    n = len(subsets)
    
    ### Build the graph of the instance
    edges = edges_from_sets(subsets)
    G = rx.PyGraph()
    G.add_nodes_from([i for i in range(1, n+1)])
    G.extend_from_edge_list(edges)
    # G.remove_node(0)
    if draw_graph:
        mpl_draw(
            G, pos=rx.circular_layout(G), with_labels=True, node_color="#EE5396", font_color="#F4F4F4"
        )
    
    ### Find the list of intersections: for every subset see
    ### with which subsets it has a non zero intersection.
    list_of_intersections = find_intersections_lists(subsets)
    
    # for i,l in enumerate(list_of_intersections):
    #     print(f"Set #{i} intersections:", l)
    # print("list_of_intersections", list_of_intersections)  

    if verbose:
        ### Find the lengths: for every subset see
        ### many elements it has.
        lengths = [len(s) for s in subsets]
        print("lenghts : ", lengths)
        
        ### Find the valencies: for every subset see
        ### with how many subsets it has a non zero intersection.
        valencies = [len(x) for x in list_of_intersections]
        print("valencies : ", valencies)

    return list_of_intersections

In [ ]:
def find_intersections_lists(subsets):
    """Parameters
       ----------
       subsets (list of sets): sets for which we want to find intersections.

       Return
       ------
       list_of_intersections (list of lists): the i-th element of the list is 
                    a list containing the indeces of all the sets which have 
                    non-zero intersection with the i-th set of `subsets`.
       
       Example
       -------
       "Set #0 intersections: [1,5]" means that the 0th set has
       nonzero intersection with the 1st and 5th set of `subsets`.
                   
    """
    list_of_intersections = []
    for i in range(len(subsets)):
        ith_set_intersections = []
        for j in range(len(subsets)):
            if i!=j and subsets[i].intersection(subsets[j]):
                ith_set_intersections.append(j)
        list_of_intersections.append(ith_set_intersections)
    return list_of_intersections

# ***************************************************************

def edges_from_sets(subsets):
    """Parameters
       ----------
       subsets (list): a list of sets.

       Returns
       -------
       egs (list of tuples): list of tuples that will be edges of a graph
                             connecting sets that have nonzero intersection.
                             E.g. (1,3) <-> subsets 1 and 3 have nonzero intersection.
    """
    egs = []
    l = len(subsets)
    for i in range(l):
        for j in range(i,l):
            w = subsets[i].intersection(subsets[j])
            if len(w)!=0 and i!=j:
                egs.append((i,j))
                
    return egs

def find_U_from_subsets(subsets_dict):
    U = subsets_dict[0]
    for s in subsets_dict.values():
        U = U | s
    return U

## Energy spectrum

In [ ]:
def bit_gen(n):
    """
    Arguments
    ---------
       n: length of the tuples to be produced.
       
    Return
    ------
       A generator of all possible n-tuples of bits.
    """
    return itertools.product([0, 1], repeat=n)

In [ ]:
def compute_energy_Lucas(x, U, subsets_dict):
    """
    Arguments:
        x (str): a binary array.
        U (set): a set.
        subsets_dict (dict): a list containing subsets of U, whose union is U.

    Return:
        E : x's energy (See "Ising formulations of many NP problems" by Andrew Lucas)
    """
    # Turn the string into list: "110000" -> [1,1,0,0,0,0]
    x = [int(digit) for digit in x]

    A = 1.

    E_A = 0.
    for uu in U:
        counts = sum([x[i] for i in subsets_dict.keys() if uu in subsets_dict[i]])
        E_A += (1 - counts)**2

    E_A = A * E_A

    return E_A


def compute_energy_Wang(state, U, subsets_dict, k=1):
    """
    Arguments:
        state : a binary array.
        U : a set.
        subsets_dict : a dictionary containing subsets of U, whose union is U.
        k (int): the multiplicative factor to compute l1 = k*len(state)*l2. It must be k>0.

    Return:
        E : state's energy (See "Quantum Alternating Operator Ansatz for Solving the Minimum Exact Cover
                             Problem" by Sha-Sha Wang et al.)
    """
    # Turn the string into list: "110000" -> [1,1,0,0,0,0]
    state = [int(digit) for digit in state]
    subsets = list(subsets_dict.values())
    # print("subsets", subsets)
    
    n = len(state)
    l2 = 1/(n*len(U) - 2)
    l1 = k * n * l2
    
    E = 0.
    for pos,digit in enumerate(state):
        w = len(subsets[pos])
        E += l1*w*digit - l2*digit

    E = -E
    return E


In [ ]:
def show_spectrum(n, instance, k, verbose=False):
    """
    Show the instance spectrum for a fixed k.

    Parameters
    ----------
        n (int): instance dimension.
        instance (int): number of the instance.
        k (float): k chosen.
        verbose (bool): if True, print is activated.
    """
    
    U, subsets_dict = define_instance(n, instance, verbose=verbose)
    
    states, energies, states_feasible, energies_feasible, EXACT_COVERS = find_spectrum(U, subsets_dict, n, k)
    MEC = [state for state in EXACT_COVERS if state.count("1") == min([x.count("1")  for x in EXACT_COVERS])]
    
    print("EXACT_COVERS:", EXACT_COVERS)
    print("MEC:", MEC)

    #############################################################################
    #############################################################################
    
    #### PLOT ALL STATES ENERGY
    dim = 20
    plt.figure(figsize=(30,5))
    plt.rcParams['font.size'] = dim
    plt.title("All states")
    plt.plot(states, energies, '.--b')
    plt.xticks(rotation='vertical', fontsize=7)
    plt.xlabel("States")
    plt.ylabel("Energy")
    # plt.ylim(-3,0)
    plt.grid()
    
    # Highlight with red the exact covers
    highlight_correct_ticks(plt.gca(), EXACT_COVERS)
    
    plt.show()
    
    
    #### PLOT THE FEASIBLE STATES ENERGIES.
    dim = 20
    plt.figure(figsize=(30,5))
    plt.rcParams['font.size'] = dim
    plt.title("Feasible states")
    plt.plot(states_feasible, energies_feasible, '.--k')
    plt.xticks(rotation='vertical', fontsize=dim)
    plt.xlabel("States")
    plt.ylabel("Energy")
    # plt.ylim(-3,0)
    plt.grid()
    
    # Highlight with red the exact covers
    highlight_correct_ticks(plt.gca(), EXACT_COVERS)
    
    plt.show()

In [13]:
def find_spectrum(U, subsets_dict, n, k):
    """
    Parameters
    ----------
        U (set): set Universe of the instance.
        subsets_dict (dict): dictionary that enumerates subsets of the instance.
        n (int): instance dimension.
        k (float): k chosen.

    Return
    ------
        states (list of str): list of all the states.
        energies (list of str): list of all the energies associated to the states.
        states_feasible (list of str): list of all the feasible states.
        energies_feasible (list of str): list of all the energies associated to the feasible states.
        EXACT_COVERS (list of str): list of all the exact covers.
    """
    subsets = list(subsets_dict.values())
    states = []
    energies = []
    states_feasible = []
    energies_feasible = []

    EXACT_COVERS = []
    
    for nuple in bit_gen(n):
        
        state = "".join([str(bit) for bit in nuple]) # strings
        chosen_subsets = [subsets[i] for i,digit in enumerate(state) if digit=="1"]
        u = set().union(*chosen_subsets)
        sum_of_len = sum([len(sub) for sub in chosen_subsets])
    
        """La somma delle lunghezze dei sottoinsiemi selezionati da uno stato
           è uguale alla lunghezza dell'insieme unione u solo 
           se i sottoinsiemi non si intersecano, 
           ovvero se lo stato soddisfa il vincolo.
        """
        energy = compute_energy_Wang(state, U, subsets_dict, k)
        
        if len(u) == sum_of_len:        
            states_feasible.append(state)
            energies_feasible.append(energy)
        
        states.append(state)
        energies.append(energy)
        
        #### CHECK IF STATE IS AN EXACT COVER
        E = compute_energy_Lucas(state, U, subsets_dict)
        if E == 0: EXACT_COVERS.append(state)
            
    return states, energies, states_feasible, energies_feasible, EXACT_COVERS
